<a href="https://colab.research.google.com/github/Jbadro/LLM-creation/blob/main/TechX2024_nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget http://gir.fyi/techx/input.txt

--2024-05-16 15:24:21--  http://gir.fyi/techx/input.txt
Resolving gir.fyi (gir.fyi)... 216.239.38.21, 216.239.36.21, 216.239.32.21, ...
Connecting to gir.fyi (gir.fyi)|216.239.38.21|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt [following]
--2024-05-16 15:24:21--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-05-16 15:24:22 (19.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
  raw_training_data = f.read()

print(len(raw_training_data))

1115394


In [4]:
len(set(raw_training_data))

65

In [5]:
''.join(sorted(set(raw_training_data)))

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [6]:
from collections import defaultdict

freqs = defaultdict(int)

for letter in raw_training_data:
  freqs[letter] += 1

freqs

defaultdict(int,
            {'F': 1797,
             'i': 45537,
             'r': 48889,
             's': 49696,
             't': 67009,
             ' ': 169892,
             'C': 3820,
             'z': 356,
             'e': 94611,
             'n': 48529,
             ':': 10316,
             '\n': 40000,
             'B': 2761,
             'f': 15770,
             'o': 65798,
             'w': 17585,
             'p': 10808,
             'c': 15623,
             'd': 31358,
             'a': 55507,
             'y': 20448,
             'u': 26584,
             'h': 51310,
             ',': 19846,
             'm': 22243,
             'k': 7088,
             '.': 7885,
             'A': 7819,
             'l': 33339,
             'S': 4523,
             'Y': 1718,
             'v': 7793,
             '?': 2462,
             'R': 4869,
             'M': 2840,
             'W': 3530,
             "'": 6187,
             'L': 3876,
             'I': 11832,
             'N': 5079,

In [7]:
import random

SAMPLE_SIZE = 500
training_data_size = len(raw_training_data)
start = random.randrange(0, training_data_size - SAMPLE_SIZE)
print(raw_training_data[start:start+SAMPLE_SIZE])

vel of the state was touch'd,
They would not thread the gates. This kind of service
Did not deserve corn gratis. Being i' the war
Their mutinies and revolts, wherein they show'd
Most valour, spoke not for them: the accusation
Which they have often made against the senate,
All cause unborn, could never be the motive
Of our so frank donation. Well, what then?
How shall this bisson multitude digest
The senate's courtesy? Let deeds express
What's like to be their words: 'we did request it;
We are th


In [9]:
tokens = ''.join(sorted(set(raw_training_data)))
token_to_number = {t: i for i, t in enumerate(tokens)}

tokens

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [10]:
number_to_token = {i: t for i, t in enumerate(tokens)}

In [11]:
def encode(s):
  return_value = []
  for character in s:
    return_value.append(token_to_number[character])
  return return_value

encode("Tech Exchange")

[32, 43, 41, 46, 1, 17, 62, 41, 46, 39, 52, 45, 43]

In [16]:
def decode(nums):
  return ''.join([number_to_token[t_id] for t_id in nums])

decode(encode("s"))


's'

In [17]:
import torch

data = torch.tensor(encode(raw_training_data), dtype=torch.long)

In [18]:
holdout_size = int(0.1 * len(data))
holdout_size

111539

In [19]:
test_data = data[:holdout_size]
training_data = data[holdout_size:]
len(test_data), len(training_data)

(111539, 1003855)

In [20]:
BLOCK_SIZE = 8
training_data[:BLOCK_SIZE+1]

tensor([43, 58,  6,  1, 25, 39, 56, 41, 47])

In [21]:
start_from = random.randint(0, training_data_size-BLOCK_SIZE)
decode(training_data[start_from:start_from+BLOCK_SIZE].tolist())

'and roya'

In [22]:
x = training_data[:BLOCK_SIZE]
y = training_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
  context = x[:t+1]
  target = y[t]
  print(f"in: {context}, out: {target}")

in: tensor([43]), out: 58
in: tensor([43, 58]), out: 6
in: tensor([43, 58,  6]), out: 1
in: tensor([43, 58,  6,  1]), out: 25
in: tensor([43, 58,  6,  1, 25]), out: 39
in: tensor([43, 58,  6,  1, 25, 39]), out: 56
in: tensor([43, 58,  6,  1, 25, 39, 56]), out: 41
in: tensor([43, 58,  6,  1, 25, 39, 56, 41]), out: 47


In [23]:
torch.manual_seed(1337)

BATCH_SIZE = 4


In [71]:
def get_batch(split):
  data = training_data if split == 'train' else test_data
  ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
  #print(ix)
  x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
  y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
  return x, y

get_batch('train')

(tensor([[56, 39, 63,  1, 63, 53, 59,  6],
         [ 1, 44, 39, 50, 57, 43,  1, 57],
         [51, 63,  1, 61, 39, 52, 58, 53],
         [56,  1, 47, 58,  1, 40, 43,  6],
         [ 1, 60, 43, 56, 63,  1, 45, 53],
         [16, 10,  0, 26, 53, 61,  6,  1],
         [ 0, 20, 53, 61,  1, 44, 39, 56],
         [44, 53, 56, 58, 59, 52, 43, 57],
         [39, 59, 52, 58,  6,  1, 58, 46],
         [56, 43, 43, 58, 50, 63,  1, 47],
         [57, 53, 51, 43,  1, 57, 46, 53],
         [ 1, 58, 43, 50, 50,  1, 51, 43],
         [47, 57,  1, 54, 39, 52, 42, 43],
         [44, 39, 50, 50,  0, 32, 46, 43],
         [58, 53,  1, 40, 43,  1, 41, 46],
         [46, 43,  1, 44, 53, 61, 50,  1],
         [ 0, 35, 46, 39, 58,  5, 57,  1],
         [21,  1, 49, 52, 53, 61,  1, 58],
         [44, 43, 41, 58, 47, 53, 52, 57],
         [58, 46, 43, 43,  1, 58, 53,  1],
         [47, 57,  1, 51, 63,  1, 57, 53],
         [39, 56,  1, 51, 43, 39, 52, 58],
         [61, 47, 58, 46,  1, 51, 63,  1],
         [5

In [72]:
xb, yb = get_batch('train')
for b in range(BATCH_SIZE):
  for t in range(BLOCK_SIZE):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'input {context.tolist()}, target {target}')

input [26], target 30
input [26, 30], target 37
input [26, 30, 37], target 1
input [26, 30, 37, 1], target 28
input [26, 30, 37, 1, 28], target 17
input [26, 30, 37, 1, 28, 17], target 30
input [26, 30, 37, 1, 28, 17, 30], target 15
input [26, 30, 37, 1, 28, 17, 30, 15], target 37
input [46], target 47
input [46, 47], target 57
input [46, 47, 57], target 1
input [46, 47, 57, 1], target 52
input [46, 47, 57, 1, 52], target 43
input [46, 47, 57, 1, 52, 43], target 61
input [46, 47, 57, 1, 52, 43, 61], target 57
input [46, 47, 57, 1, 52, 43, 61, 57], target 1
input [56], target 58
input [56, 58], target 1
input [56, 58, 1], target 56
input [56, 58, 1, 56], target 47
input [56, 58, 1, 56, 47], target 41
input [56, 58, 1, 56, 47, 41], target 46
input [56, 58, 1, 56, 47, 41, 46], target 6
input [56, 58, 1, 56, 47, 41, 46, 6], target 1
input [1], target 58
input [1, 58], target 53
input [1, 58, 53], target 1
input [1, 58, 53, 1], target 43
input [1, 58, 53, 1, 43], target 51
input [1, 58, 53,

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [68]:
vocab_size = len(set(raw_training_data))

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, token, targets=None):
    logits = self.token_embedding_table(token)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

m = BigramLanguageModel(vocab_size)
z = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(z, 40)[0].tolist()))


Q&Ol!BfXHM&-Jloy&pDkX
OV-SR&oxv.IZI
cFnt


In [73]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

BATCH_SIZE = 32

for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.439030885696411


In [78]:
z = torch.zeros((1,1), dtype=torch.long)
generated = m.generate(z, max_new_tokens=100)
generated_list = generated[0].tolist()
print(decode(generated_list))


'sed d vigomysen; w thengod, o t tix'se yod pe w had tier'
A:
S:
O:
KIVMam;
Fincy be's pp fsonengn n


In [32]:
import numpy as np

-np.log(1/65)

4.174387269895637